In [47]:
##########################################################################
# Space robot simulator (planar case: 3DoF satellite + 3DoF robotic arm) #
# Input: control torques
# Output: space robot behaviour (satellite and robotic arm trajectories) #
# Author: Fatina Liliana Basmadji  (fbasmadji@cbk.waw.pl)                #
##########################################################################

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from matplotlib import rc, animation

In [49]:
# Load control torques
control_torques = np.genfromtxt('control-torques.txt')

In [50]:
# Hyper parameters
dt = 0.01
nstep = 500

In [51]:
# Space robot parameters

pi = 3.14159
p1 = 0.3696    # manipulator mounting point in x axis
p2 = 0.001279  # manipulator mounting point in y axis
a1 = 0.1805    # center of mass of the first link
a2 = 0.2000    # center of mass of the second link
a3 = 0.1029    # center of mass of the third link
L1 = 0.4488    # the length of the first link
L2 = 0.4499    # the length of the second link
L3 = 0.3545    # the length of the third link
m0 = 64.859    # the mass of the satellite
m1 = 2.9130    # the mass of the first kinematic pair
m2 = 2.6460    # the mass of the second kinematic pair
m3 = 1.6990    # the mass of the third kinematic pair
I0 = 2.6952    # Satellite moment of inertia
I1 = 0.091391  # Moment of inertia of the first kinematic pair
I2 = 0.081375  # Moment of inertia of the second kinematic pair
I3 = 0.021904  # Moment of inertia of the third kinematic pair
d  = 0.3536    # Distance from base geometry center to each corner

In [52]:
def kinematics(state):
    EE = np.zeros(3)
    Jacobian = np.zeros((3,6))
    EE[0] = state[0] + p1*cos(state[2]) - p2*sin(state[2]) + L1*cos(state[2]+state[3]) + L2*cos(state[2]+state[3]+state[4]) + L3*cos(state[2]+state[3]+state[4]+state[5]);
    EE[1] = state[1] + p1*sin(state[2]) + p2*cos(state[2]) + L1*sin(state[2]+state[3]) + L2*sin(state[2]+state[3]+state[4]) + L3*sin(state[2]+state[3]+state[4]+state[5]);
    EE[2] = state[2] + state[3] + state[4] + state[5];
    Jacobian[0][0] = 1;
    Jacobian[0][1] = 0;
    Jacobian[0][2] = - L3*sin(state[2]+state[3]+state[4]+state[5]) - L1*sin(state[2]+state[3]) - p2*cos(state[2]) - p1*sin(state[2]) - L2*sin(state[2]+state[3]+state[4]);
    Jacobian[0][3] = - L3*sin(state[2]+state[3]+state[4]+state[5]) - L1*sin(state[2]+state[3]) - L2*sin(state[2]+state[3]+state[4]);
    Jacobian[0][4] = - L3*sin(state[2]+state[3]+state[4]+state[5]) - L2*sin(state[2]+state[3]+state[4]);
    Jacobian[0][5] = - L3*sin(state[2]+state[3]+state[4]+state[5]);
    Jacobian[1][0] = 0;
    Jacobian[1][1] = 1;
    Jacobian[1][2] = L3*cos(state[2]+state[3]+state[4]+state[5]) + L1*cos(state[2]+state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2]+state[3]+state[4]);
    Jacobian[1][3] = L3*cos(state[2]+state[3]+state[4]+state[5]) + L1*cos(state[2]+state[3]) + L2*cos(state[2]+state[3]+state[4]);
    Jacobian[1][4] = L3*cos(state[2]+state[3]+state[4]+state[5]) + L2*cos(state[2]+state[3]+state[4]);
    Jacobian[1][5] = L3*cos(state[2]+state[3]+state[4]+state[5]);
    Jacobian[2][0] = 0;
    Jacobian[2][1] = 0;
    Jacobian[2][2] = 1;
    Jacobian[2][3] = 1;
    Jacobian[2][4] = 1;
    Jacobian[2][5] = 1;
    return EE, Jacobian

In [53]:
def dynamics(state):
    J0v = np.zeros((2,6))
    J0w = np.zeros((1,6))
    J1v = np.zeros((2,6))
    J1w = np.zeros((1,6))
    J2v = np.zeros((2,6))
    J2w = np.zeros((1,6))
    J3v = np.zeros((2,6))
    J3w = np.zeros((1,6))
    C = np.zeros((6,1))
    J0v = [[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]]
    J0vt = np.transpose(J0v)
    J0w = [[0, 0, 1, 0, 0, 0]]
    J0wt = np.transpose(J0w)
    var1 = - p1*sin(state[2]) -p2*cos(state[2]) - a1*sin(state[2]+state[3])
    var2 = -a1*sin(state[2]+state[3])
    var3 = -p2*sin(state[2]) + p1*cos(state[2])+a1*cos(state[3]+state[2])
    var4 = a1*cos(state[3]+state[2])
    J1v = [[1, 0, var1, var2, 0, 0],[0, 1, var3, var4, 0, 0]];
    J1vt = np.transpose(J1v)  
    J1w = [[0, 0, 1, 1, 0, 0]]
    J1wt = np.transpose(J1w)
    var5 = -p2*cos(state[2]) - L1*sin(state[3]+state[2]) - p1*sin(state[2]) - a2*sin(state[3]+state[4]+state[2]);
    var6 = -L1*sin(state[3]+state[2]) - a2*sin(state[3]+state[4]+state[2]);
    var7 = -a2*sin(state[3]+state[4]+state[2]);
    var8 = -p2*sin(state[2]) + a2*cos(state[3]+state[4]+state[2]) + L1*cos(state[3]+state[2])+p1*cos(state[2]);
    var9 =  a2*cos(state[3]+state[4]+state[2]) + L1*cos(state[3]+state[2]);
    var10 = a2*cos(state[3]+state[4]+state[2]);
    J2v = [[1, 0, var5, var6, var7, 0],[0, 1, var8, var9, var10, 0]]   
    J2vt = np.transpose(J2v)  
    J2w = [[0, 0, 1, 1, 1, 0]]
    J2wt = np.transpose(J2w)
    var11 = -p2*cos(state[2]) - L1*sin(state[3]+state[2]) - p1*sin(state[2]) - L2*sin(state[3]+state[4]+state[2]) -a3*sin(state[2]+state[3]+state[4]+state[5]);
    var12 = -L1*sin(state[3]+state[2]) - L2*sin(state[3]+state[4]+state[2]) - a3*sin(state[3]+state[4]+state[2]+state[5]);
    var13 = -L2*sin(state[3]+state[4]+state[2]) - a3*sin(state[3]+state[4]+state[2]+state[5]);
    var14 = -a3*sin(state[3]+state[4]+state[2]+state[5]);
    var15 = -p2*sin(state[2]) + L2*cos(state[3]+state[4]+state[2]) + L1*cos(state[3]+state[2])+p1*cos(state[2]) + a3*cos(state[2]+state[3]+state[4]+state[5]);
    var16 =  L2*cos(state[3]+state[4]+state[2]) + L1*cos(state[3]+state[2]) + a3*cos(state[2]+state[3]+state[4]+state[5]);
    var17 =  L2*cos(state[3]+state[4]+state[2]) + a3*cos(state[2]+state[3]+state[4]+state[5]);
    var18 =  a3*cos(state[2]+state[3]+state[4]+state[5]);
    J3v = [[1, 0, var11, var12, var13, var14],[0, 1, var15, var16, var17, var18]]
    J3vt = np.transpose(J3v)
    J3w = [[0, 0, 1, 1, 1, 1]]
    J3wt = np.transpose(J3w)
    Tv = np.matmul(m0*J0vt,J0v) +  np.matmul(m1*J1vt,J1v) +  np.matmul(m2*J2vt,J2v) +  np.matmul(m3*J3vt,J3v)
    Tw = J0wt*I0*J0w +  J1wt*I1*J1w +  J2wt*I2*J2w + J3wt*I3*J3w;
    M  = Tv+Tw;
    C[0][0] = - state[10]*(state[8]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[9]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[10]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + a3*m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])) - state[8]*(state[8]*(m2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m1*(a1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]))) + state[10]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[9]*(m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + a1*m1*cos(state[2] + state[3])) + a3*m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])) - state[9]*(state[10]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[8]*(m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + a1*m1*cos(state[2] + state[3])) + state[9]*(m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + a1*m1*cos(state[2] + state[3])) + a3*m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])) - state[11]*(a3*m3*state[8]*cos(state[2] + state[3] + state[4] + state[5]) + a3*m3*state[9]*cos(state[2] + state[3] + state[4] + state[5]) + a3*m3*state[10]*cos(state[2] + state[3] + state[4] + state[5]) + a3*m3*state[11]*cos(state[2] + state[3] + state[4] + state[5]));
    C[1][0] = - state[11]*(a3*m3*state[8]*sin(state[2] + state[3] + state[4] + state[5]) + a3*m3*state[9]*sin(state[2] + state[3] + state[4] + state[5]) + a3*m3*state[10]*sin(state[2] + state[3] + state[4] + state[5]) + a3*m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])) - state[8]*(state[9]*(m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a1*m1*sin(state[2] + state[3])) + state[10]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + state[8]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + m2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) + m1*(a1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]))) + a3*m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])) - state[9]*(state[8]*(m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a1*m1*sin(state[2] + state[3])) + state[9]*(m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a1*m1*sin(state[2] + state[3])) + state[10]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + a3*m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])) - state[10]*(state[8]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + state[9]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + state[10]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + a3*m3*state[11]*sin(state[2] + state[3] + state[4] + state[5]));
    C[2][0] = (state[9]*(state[6]*(m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + a1*m1*cos(state[2] + state[3])) + state[7]*(m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a1*m1*sin(state[2] + state[3]))))/2 - state[8]*(state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) + state[10]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4]))) - state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) - m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4]))*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m1*cos(state[2] + state[3])*a1*(a1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2])) - m1*sin(state[2] + state[3])*a1*(a1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2])) + a1*m1*cos(state[2] + state[3])*(cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*a1) - a1*m1*sin(state[2] + state[3])*(cos(state[2])*p1 - sin(state[2])*p2 + cos(state[2] + state[3])*a1))) - state[6]*(state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m1*cos(state[2] + state[3])*a1) + state[10]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[8]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m1*(cos(state[2])*p1 - sin(state[2])*p2 + cos(state[2] + state[3])*a1) + m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2)) + m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])*a3) - state[7]*(state[8]*(m2*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m1*(cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*a1)) + state[9]*(m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m1*sin(state[2] + state[3])*a1) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])*a3) + (state[11]*(a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 - state[11]*(state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[10]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)) + state[9]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1))) - state[9]*(state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))) - state[9]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4]))*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) - m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + a1*m1*cos(state[2] + state[3])*(cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*a1) - a1*m1*sin(state[2] + state[3])*(cos(state[2])*p1 - sin(state[2])*p2 + cos(state[2] + state[3])*a1) + a1*m1*cos(state[2] + state[3])*sin(state[2] + state[3])*a1 - a1*m1*sin(state[2] + state[3])*cos(state[2] + state[3])*a1) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1))) + (state[10]*(state[6]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[7]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4]))))/2 - state[10]*(state[9]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2) + state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])))) + (state[8]*(state[6]*(m2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m1*(a1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]))) + state[7]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + m2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) + m1*(a1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2])))))/2 + (state[6]*(state[10]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[8]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m1*(cos(state[2])*p1 - sin(state[2])*p2 + cos(state[2] + state[3])*a1) + m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2)) + state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m1*cos(state[2] + state[3])*a1) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 + (state[7]*(state[9]*(m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m1*sin(state[2] + state[3])*a1) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + state[8]*(m2*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m1*(cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*a1)) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2;
    C[3][0] = (state[10]*(state[6]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[7]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + state[8]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1))))/2 - state[11]*(state[10]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)) + state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3)) - state[10]*(state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) - a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2) + state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1))) - (state[8]*(state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) - state[6]*(m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + a1*m1*cos(state[2] + state[3])) + state[8]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) - m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4]))*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m1*cos(state[2] + state[3])*a1*(a1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2])) - m1*sin(state[2] + state[3])*a1*(a1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2])) + a1*m1*cos(state[2] + state[3])*(cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*a1) - a1*m1*sin(state[2] + state[3])*(cos(state[2])*p1 - sin(state[2])*p2 + cos(state[2] + state[3])*a1)) + state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) - m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m1*cos(state[2] + state[3])*a1*(a1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2])) - m1*sin(state[2] + state[3])*a1*(a1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2])) - a1*m1*cos(state[2] + state[3])*sin(state[2] + state[3])*a1 + a1*m1*sin(state[2] + state[3])*cos(state[2] + state[3])*a1) - state[7]*(m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a1*m1*sin(state[2] + state[3])) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) - m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])))))/2 + (state[6]*(state[10]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[8]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m1*cos(state[2] + state[3])*a1) + state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m1*cos(state[2] + state[3])*a1) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 + (state[7]*(state[8]*(m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m1*sin(state[2] + state[3])*a1) + state[9]*(m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m1*sin(state[2] + state[3])*a1) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 + (state[11]*(state[8]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 - state[9]*(state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) - a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1))) - state[6]*(state[8]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m1*cos(state[2] + state[3])*a1) + state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) + m1*cos(state[2] + state[3])*a1) + state[10]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])*a3) - state[7]*(state[8]*(m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m1*sin(state[2] + state[3])*a1) + state[9]*(m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m1*sin(state[2] + state[3])*a1) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])*a3) - state[8]*(state[10]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) - a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4]))) - state[9]*(m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) - m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m1*cos(state[2] + state[3])*a1*(a1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2])) - m1*sin(state[2] + state[3])*a1*(a1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2])) - a1*m1*cos(state[2] + state[3])*sin(state[2] + state[3])*a1 + a1*m1*sin(state[2] + state[3])*cos(state[2] + state[3])*a1) + state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])))) + (state[9]*(state[6]*(m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + a1*m1*cos(state[2] + state[3])) + state[7]*(m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a1*m1*sin(state[2] + state[3])) - state[8]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4]))*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m2*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1)*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) - m2*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + a1*m1*cos(state[2] + state[3])*(cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*a1) - a1*m1*sin(state[2] + state[3])*(cos(state[2])*p1 - sin(state[2])*p2 + cos(state[2] + state[3])*a1) + a1*m1*cos(state[2] + state[3])*sin(state[2] + state[3])*a1 - a1*m1*sin(state[2] + state[3])*cos(state[2] + state[3])*a1)))/2;
    C[4][0] = (state[8]*(state[6]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) - state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) - m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2) + state[8]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4]))) + state[7]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) - state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) - m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) + state[9]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) - a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])))))/2 - state[7]*(state[8]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + state[9]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])*a3) + state[9]*(state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2) - state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2))) - state[11]**2*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + (state[9]*(state[6]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[7]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + state[9]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) - a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1)) - state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2) - state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) - m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))) + state[8]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1))))/2 - state[6]*(state[8]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[9]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[10]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])*a3) + (state[7]*(state[8]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + state[9]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m2*sin(state[2] + state[3] + state[4])*a2) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 + state[8]*(state[9]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + a2*cos(state[2] + state[3] + state[4])) - m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + a2*sin(state[2] + state[3] + state[4])) - m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4]))) - state[11]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) + state[10]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2)*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) - m2*sin(state[2] + state[3] + state[4])*a2*(L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + a2*cos(state[2] + state[3] + state[4])) + m2*cos(state[2] + state[3] + state[4])*a2*(L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + a2*sin(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2)) + (state[10]*(state[6]*(m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a2*m2*cos(state[2] + state[3] + state[4])) + state[7]*(m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a2*m2*sin(state[2] + state[3] + state[4])) + state[9]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2] + state[3])*L1) - a2*m2*cos(state[2] + state[3] + state[4])*(sin(state[2] + state[3] + state[4])*a2 + sin(state[2] + state[3])*L1) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2) + state[8]*(m3*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - m3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4]))*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + a2*m2*sin(state[2] + state[3] + state[4])*(cos(state[2] + state[3] + state[4])*a2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a2*m2*cos(state[2] + state[3] + state[4])*(cos(state[2])*p2 + sin(state[2] + state[3] + state[4])*a2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + a2*m2*cos(state[2] + state[3] + state[4])*sin(state[2] + state[3] + state[4])*a2 - a2*m2*sin(state[2] + state[3] + state[4])*cos(state[2] + state[3] + state[4])*a2)))/2 + (state[11]*(state[8]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)) + state[9]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 + (state[6]*(state[8]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[9]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + state[10]*(m3*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) + m2*cos(state[2] + state[3] + state[4])*a2) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 - state[10]*state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2));
    C[5][0] = (state[10]*(state[8]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4]))) + state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[10]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)) + state[9]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1)) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 - state[9]*(state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) - state[10]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) - m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])))) + (state[6]*(m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[8] + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[9] + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[10] + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 + state[8]*(state[10]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) - m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) - state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[9]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) - m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) + m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])))) + (state[7]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[8] + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[9] + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[10] + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*state[11]))/2 - state[6]*(m3*state[8]*cos(state[2] + state[3] + state[4] + state[5])*a3 + m3*state[9]*cos(state[2] + state[3] + state[4] + state[5])*a3 + m3*state[10]*cos(state[2] + state[3] + state[4] + state[5])*a3 + m3*state[11]*cos(state[2] + state[3] + state[4] + state[5])*a3) + (state[8]*(state[8]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) + state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[10]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) + state[9]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + p1*cos(state[2]) - p2*sin(state[2]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + p2*cos(state[2]) + p1*sin(state[2]) + L2*sin(state[2] + state[3] + state[4]))) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 - state[7]*(m3*state[8]*sin(state[2] + state[3] + state[4] + state[5])*a3 + m3*state[9]*sin(state[2] + state[3] + state[4] + state[5])*a3 + m3*state[10]*sin(state[2] + state[3] + state[4] + state[5])*a3 + m3*state[11]*sin(state[2] + state[3] + state[4] + state[5])*a3) + (state[11]*(state[8]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[10]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[9]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 + (state[9]*(state[8]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2])*p1 + cos(state[2] + state[3])*L1 - sin(state[2])*p2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + cos(state[2])*p2 + sin(state[2])*p1 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))) + state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3) + state[10]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2)) + state[9]*(a3*m3*sin(state[2] + state[3] + state[4] + state[5])*(cos(state[2] + state[3] + state[4] + state[5])*a3 + cos(state[2] + state[3] + state[4])*L2 + cos(state[2] + state[3])*L1) - a3*m3*cos(state[2] + state[3] + state[4] + state[5])*(sin(state[2] + state[3] + state[4] + state[5])*a3 + sin(state[2] + state[3] + state[4])*L2 + sin(state[2] + state[3])*L1) + m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L1*cos(state[2] + state[3]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L1*sin(state[2] + state[3]) + L2*sin(state[2] + state[3] + state[4]))) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*state[6] + a3*m3*sin(state[2] + state[3] + state[4] + state[5])*state[7]))/2 - state[10]*state[11]*(m3*sin(state[2] + state[3] + state[4] + state[5])*a3*(a3*cos(state[2] + state[3] + state[4] + state[5]) + L2*cos(state[2] + state[3] + state[4])) - m3*cos(state[2] + state[3] + state[4] + state[5])*a3*(a3*sin(state[2] + state[3] + state[4] + state[5]) + L2*sin(state[2] + state[3] + state[4])) + a3*m3*cos(state[2] + state[3] + state[4] + state[5])*sin(state[2] + state[3] + state[4] + state[5])*a3 - a3*m3*sin(state[2] + state[3] + state[4] + state[5])*cos(state[2] + state[3] + state[4] + state[5])*a3);
    return M, C

In [54]:
# Initial conditions

SatPosX = 0           # x-positon of the satellite's center of mass [m]
SatPosY = 0           # y-positon of the satellite's center of mass [m]
SatOrient = 0         # satellite orientation [rad]
J1AngPos = -1.0882    # angular position of the first joint [rad]
J2AngPos = 0.7349     # angular position of the second joint [rad]
J3AngPos = 1.9241     # angular position of the third joint [rad]
SatVelX = 0           # x-velocity of the satellite's center of mass [m/sec]
SatVelY = 0           # y-velocity of the satellite's center of mass [m/sec]
SatAngVel = 0         # satellite angular velocity [rad/sec]
J1AngVel = 0          # angular velocity of the first joint [rad/sec]
J2AngVel = 0          # angular velocity of the second joint [rad/sec]
J3AngVel = 0          # angular velocity of the third joint [rad/sec]

In [55]:
# Simulating space robot behaviour

x = np.array([SatPosX,SatPosY,SatOrient,J1AngPos,J2AngPos,J3AngPos,SatVelX,SatVelY,SatAngVel,J1AngVel,J2AngVel,J3AngVel])

q  = np.array([[0], [0], [0], [-1.0882], [0.7349], [1.9241]])
dq = np.array([[0], [0], [0], [0], [0], [0]])


sat_pos_x = np.zeros(nstep)
sat_pos_y = np.zeros(nstep)
sat_orient = np.zeros(nstep)
joint_1 = np.zeros(nstep)
joint_2 = np.zeros(nstep)
joint_3 = np.zeros(nstep)
F = np.zeros((6,1))
EE = np.zeros((nstep,3))

for j in range(0,nstep):
    if j == 0:
        EE[0,:], Jacobian = kinematics(x)
        sat_pos_x[0] = x[0]
        sat_pos_y[0] = x[1]
        sat_orient[0] = x[2]
        joint_1[0] = x[3]
        joint_2[0] = x[4]
        joint_3[0] = x[5]
    else:
        F = [[0],[0],[0],[control_torques[j-1][8]], [control_torques[j-1][9]], [control_torques[j-1][10]]]
        M, C = dynamics(x)
        k11 = dt*dq;
        k21 = dt*np.matmul(np.linalg.inv(M),(F-C))
        k12 = dt*(dq+0.5*k21)
        u= dq+0.5*k21
        x[6] = u[0]
        x[7] = u[1]
        x[8] = u[2]
        x[9] = u[3]
        x[10] = u[4]
        x[11] = u[5]
        M, C = dynamics(x);
        k22 = dt*np.matmul(np.linalg.inv(M),(F-C))
        k13 = dt*(dq+0.5*k22)
        u = dq+0.5*k22
        x[6] = u[0]
        x[7] = u[1]
        x[8] = u[2]
        x[9] = u[3]
        x[10] = u[4]
        x[11] = u[5]
        M, C = dynamics(x)
        k23 = dt*np.matmul(np.linalg.inv(M),(F-C))
        k14 = dt*(dq+k23)
        u = dq+k23
        x[6] = u[0]
        x[7] = u[1]
        x[8] = u[2]
        x[9] = u[3]
        x[10] = u[4]
        x[11] = u[5]
        M, C = dynamics(x)
        k24 = dt*np.matmul(np.linalg.inv(M),(F-C))
        q = q + (k11+2*k12+2*k13+k14)/6
        dq = dq + (k21+2*k22+2*k23+k24)/6
        x = [q[0], q[1],  q[2],  q[3],  q[4],  q[5], dq[0], dq[1], dq[2], dq[3], dq[4], dq[5]]  
        sat_pos_x[j] = x[0]
        sat_pos_y[j] = x[1]
        sat_orient[j] = x[2]
        joint_1[j] = x[3]
        joint_2[j] = x[4]
        joint_3[j] = x[5] 
        EE[j,:], Jacobian = kinematics(x)    

In [56]:
# Some calculations needed for animation and collision detection

BaseXwn = np.zeros(500)
BaseYwn = np.zeros(500)
BaseXws = np.zeros(500)
BaseYws = np.zeros(500) 
BaseXen = np.zeros(500)
BaseYen = np.zeros(500)
BaseXes = np.zeros(500)
BaseYes = np.zeros(500)
Link1Xws = np.zeros(500) 
Link1Yws = np.zeros(500) 
Link1Xwn = np.zeros(500) 
Link1Ywn = np.zeros(500) 
Link1Xes = np.zeros(500) 
Link1Yes = np.zeros(500) 
Link1Xen = np.zeros(500) 
Link1Yen = np.zeros(500) 
MPXws = np.zeros(500)
MPYws = np.zeros(500)
MPXwn = np.zeros(500)
MPYwn = np.zeros(500)
MPXes = np.zeros(500)
MPYes = np.zeros(500)
MPXen = np.zeros(500)
MPYen = np.zeros(500)
Link2Xws = np.zeros(500) 
Link2Yws = np.zeros(500) 
Link2Xwn = np.zeros(500) 
Link2Ywn = np.zeros(500) 
Link2Xes = np.zeros(500) 
Link2Yes = np.zeros(500) 
Link2Xen = np.zeros(500) 
Link2Yen = np.zeros(500) 
Link3Xws = np.zeros(500) 
Link3Yws = np.zeros(500) 
Link3Xwn = np.zeros(500) 
Link3Ywn = np.zeros(500) 
Link3Xes = np.zeros(500) 
Link3Yes = np.zeros(500) 
Link3Xen = np.zeros(500) 
Link3Yen = np.zeros(500) 
MPL1X = np.zeros(500)
MPL1Y = np.zeros(500)
L1L2X = np.zeros(500)
L1L2Y = np.zeros(500)
L2L3X = np.zeros(500)
L2L3Y = np.zeros(500)

satellite_edge = 0.5

d1 = 0.25
d2 = 0.025

for j in range(0,500):
    satWN = [[sat_pos_x[j]-satellite_edge/2, sat_pos_y[j] +satellite_edge/2]]
    satWS = [[sat_pos_x[j]-satellite_edge/2, sat_pos_y[j] -satellite_edge/2]]
    satEN = [[sat_pos_x[j]+satellite_edge/2, sat_pos_y[j] +satellite_edge/2]]
    satES = [[sat_pos_x[j]+satellite_edge/2, sat_pos_y[j] -satellite_edge/2]]
    rotation_matrix = [[cos(sat_orient[j]), sin(sat_orient[j])], [-sin(sat_orient[j]), cos(sat_orient[j])]]
    rotation_matrix = np.transpose(rotation_matrix)
    satwn = np.matmul(rotation_matrix,np.transpose(np.subtract(satWN,[[sat_pos_x[j], sat_pos_y[j]]])))
    satws = np.matmul(rotation_matrix,np.transpose(np.subtract(satWS,[[sat_pos_x[j], sat_pos_y[j]]])))
    saten = np.matmul(rotation_matrix,np.transpose(np.subtract(satEN,[[sat_pos_x[j], sat_pos_y[j]]])))
    sates = np.matmul(rotation_matrix,np.transpose(np.subtract(satES,[[sat_pos_x[j], sat_pos_y[j]]])))
    satWN = np.add(np.transpose(satwn), [[sat_pos_x[j], sat_pos_y[j]]])
    satWS = np.add(np.transpose(satws), [[sat_pos_x[j], sat_pos_y[j]]])
    satEN = np.add(np.transpose(saten), [[sat_pos_x[j], sat_pos_y[j]]])
    satES = np.add(np.transpose(sates), [[sat_pos_x[j], sat_pos_y[j]]])
    BaseXwn[j] = satWN[0][0]
    BaseYwn[j] = satWN[0][1]
    BaseXws[j] = satWS[0][0]
    BaseYws[j] = satWS[0][1]    
    BaseXen[j] = satEN[0][0]
    BaseYen[j] = satEN[0][1]   
    BaseXes[j] = satES[0][0]
    BaseYes[j] = satES[0][1]
    MPXws[j] = sat_pos_x[j] + d1*cos(sat_orient[j]) - (p2-d2)*sin(sat_orient[j])
    MPYws[j] = sat_pos_y[j] + d1*sin(sat_orient[j]) + (p2-d2)*cos(sat_orient[j])
    MPXwn[j] = sat_pos_x[j] + d1*cos(sat_orient[j]) - (d2+p2)*sin(sat_orient[j])
    MPYwn[j] = sat_pos_y[j] + d1*sin(sat_orient[j]) + (d2+p2)*cos(sat_orient[j])
    MPXes[j] = MPXws[j] + (p1-0.25)*cos(sat_orient[j])
    MPYes[j] = MPYws[j] + (p1-0.25)*sin(sat_orient[j])
    MPXen[j] = MPXwn[j] + (p1-0.25)*cos(sat_orient[j])
    MPYen[j] = MPYwn[j] + (p1-0.25)*sin(sat_orient[j])
    MPL1X[j] = (MPXes[j]+MPXen[j])/2
    MPL1Y[j] = (MPYes[j]+MPYen[j])/2
    Link1Xws[j] = MPL1X[j] + d2*sin(sat_orient[j]+joint_1[j])
    Link1Yws[j] = MPL1Y[j] - d2*cos(sat_orient[j]+joint_1[j])
    Link1Xwn[j] = MPL1X[j] - d2*sin(sat_orient[j]+joint_1[j])
    Link1Ywn[j] = MPL1Y[j] + d2*cos(sat_orient[j]+joint_1[j])
    Link1Xes[j] = Link1Xws[j] + L1*cos(sat_orient[j]+joint_1[j])
    Link1Yes[j] = Link1Yws[j] + L1*sin(sat_orient[j]+joint_1[j])
    Link1Xen[j] = Link1Xwn[j] + L1*cos(sat_orient[j]+joint_1[j])
    Link1Yen[j] = Link1Ywn[j] + L1*sin(sat_orient[j]+joint_1[j])
    L1L2X[j] = (Link1Xes[j]+Link1Xen[j])/2
    L1L2Y[j] = (Link1Yes[j]+Link1Yen[j])/2
    Link2Xws[j] = L1L2X[j] + d2*sin(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Yws[j] = L1L2Y[j] - d2*cos(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Xwn[j] = L1L2X[j] - d2*sin(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Ywn[j] = L1L2Y[j] + d2*cos(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Xes[j] = Link2Xws[j] + L2*cos(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Yes[j] = Link2Yws[j] + L2*sin(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Xen[j] = Link2Xwn[j] + L2*cos(sat_orient[j]+joint_1[j]+joint_2[j])
    Link2Yen[j] = Link2Ywn[j] + L2*sin(sat_orient[j]+joint_1[j]+joint_2[j])    
    L2L3X[j] = (Link2Xes[j]+Link2Xen[j])/2
    L2L3Y[j] = (Link2Yes[j]+Link2Yen[j])/2
    Link3Xws[j] = L2L3X[j] + d2*sin(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Yws[j] = L2L3Y[j] - d2*cos(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Xwn[j] = L2L3X[j] - d2*sin(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Ywn[j] = L2L3Y[j] + d2*cos(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Xes[j] = Link3Xws[j] + L3*cos(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Yes[j] = Link3Yws[j] + L3*sin(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Xen[j] = Link3Xwn[j] + L3*cos(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])
    Link3Yen[j] = Link3Ywn[j] + L3*sin(sat_orient[j]+joint_1[j]+joint_2[j]+joint_3[j])

In [58]:
# Animation

%matplotlib qt

fig = plt.figure()

ax = fig.add_subplot(111)

plt.axis('equal')

ax.set_xlim(-1, 2)
ax.set_ylim(-1, 1)

xdata, ydata = [], []
ee, = plt.plot([], [], 'r', animated=False)

Base  = np.array([[BaseXws[0],BaseYws[0]],[BaseXes[0],BaseYes[0]],[BaseXen[0],BaseYen[0]],[BaseXwn[0],BaseYwn[0]],[BaseXws[0],BaseYws[0]]])
MP = np.array([[MPXws[0],MPYws[0]], [MPXes[0],MPYes[0]], [MPXen[0],MPYen[0]], [MPXwn[0],MPYwn[0]], [MPXws[0],MPYws[0]]])
Link1 = np.array([[Link1Xws[0],Link1Yws[0]], [Link1Xes[0],Link1Yes[0]], [Link1Xen[0],Link1Yen[0]], [Link1Xwn[0],Link1Ywn[0]], [Link1Xws[0],Link1Yws[0]]])
Link2 = np.array([[Link2Xws[0],Link2Yws[0]], [Link2Xes[0],Link2Yes[0]], [Link2Xen[0],Link2Yen[0]], [Link2Xwn[0],Link2Ywn[0]], [Link2Xws[0],Link2Yws[0]]])
Link3 = np.array([[Link3Xws[0],Link3Yws[0]], [Link3Xes[0],Link3Yes[0]], [Link3Xen[0],Link3Yen[0]], [Link3Xwn[0],Link3Ywn[0]], [Link3Xws[0],Link3Yws[0]]])

kwds = dict(ec='k', alpha=0.5)
patches = [ax.add_patch(plt.Polygon(0 * Base, fc='b', **kwds)),
           ax.add_patch(plt.Polygon(0 * MP, fc='b', **kwds)),
           ax.add_patch(plt.Polygon(0 * Link1, fc='darkkhaki', **kwds)),
           ax.add_patch(plt.Polygon(0 * Link2, fc='darkkhaki', **kwds)),
           ax.add_patch(plt.Polygon(0 * Link3, fc='darkkhaki', **kwds)),
           ax.add_patch(plt.Circle((0,0),0.04, fc="dimgrey")),
           ax.add_patch(plt.Circle((0,0),0.04, fc="dimgrey")),
           ax.add_patch(plt.Circle((0,0),0.04, fc="dimgrey"))]



def SpaRobAnim(i):
    temp1 = [[BaseXws[i],BaseYws[i]],[BaseXes[i],BaseYes[i]],[BaseXen[i],BaseYen[i]],[BaseXwn[i],BaseYwn[i]],[BaseXws[i],BaseYws[i]]]
    patches[0].set_xy(temp1)
    temp2 = [[MPXws[i],MPYws[i]], [MPXes[i],MPYes[i]], [MPXen[i],MPYen[i]], [MPXwn[i],MPYwn[i]], [MPXws[i],MPYws[i]]]
    patches[1].set_xy(temp2)
    temp3 = [[Link1Xws[i],Link1Yws[i]], [Link1Xes[i],Link1Yes[i]], [Link1Xen[i],Link1Yen[i]], [Link1Xwn[i],Link1Ywn[i]], [Link1Xws[i],Link1Yws[i]]]
    patches[2].set_xy(temp3)
    temp4 = [[Link2Xws[i],Link2Yws[i]], [Link2Xes[i],Link2Yes[i]], [Link2Xen[i],Link2Yen[i]], [Link2Xwn[i],Link2Ywn[i]], [Link2Xws[i],Link2Yws[i]]]
    patches[3].set_xy(temp4)
    temp5 = [[Link3Xws[i],Link3Yws[i]], [Link3Xes[i],Link3Yes[i]], [Link3Xen[i],Link3Yen[i]], [Link3Xwn[i],Link3Ywn[i]], [Link3Xws[i],Link3Yws[i]]]
    patches[4].set_xy(temp5)
    patches[5].center = (MPL1X[i],MPL1Y[i])
    patches[6].center = (L1L2X[i],L1L2Y[i])
    patches[7].center = (L2L3X[i],L2L3Y[i])
    xdata.append(EE[i,0])
    ydata.append(EE[i,1])
    ee.set_data(xdata,ydata)
    return patches, ee


def init():
    ee.set_data(xdata,ydata)
    return ee


anim = animation.FuncAnimation(fig, SpaRobAnim,
                               frames=np.arange(0,len(BaseYes),1),
                               interval=10, blit=False, repeat=False,init_func=init)